In [1]:
from langchain.tools import StructuredTool
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import LLMSingleActionAgent
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import AgentExecutor


from tools.tools import (
    google_place_search_tool,
    search_url_tool,
    request_url_tool,
    savefile_tool,
    readfile_tool,
    translation_tool
)
from prompts.template import ENG_TEMPLATE
from prompts.purpose import ENG_PURPOSE
from utils.customization import CustomPromptTemplate


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
tools = [
    StructuredTool.from_function(google_place_search_tool),
    StructuredTool.from_function(search_url_tool),
    StructuredTool.from_function(request_url_tool),
    # StructuredTool.from_function(savefile_tool),
    # StructuredTool.from_function(readfile_tool),
    StructuredTool.from_function(request_url_tool),
    StructuredTool.from_function(translation_tool),
]
tool_names = [tool.name for tool in tools]

prompt = CustomPromptTemplate(
    template=ENG_TEMPLATE,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

llm = ChatOpenAI(temperature=0, model="gpt-4")
llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=JSONAgentOutputParser(),
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

memory = ConversationBufferWindowMemory(k=4)


agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    max_iterations=64,
    handle_parsing_errors=True
)

agent_executor.run(ENG_PURPOSE.format(query="방배동 트릿지 근처에 고기 맛있는 곳 어디야?"))

/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.agent.LLMSingleActionAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecation



> Entering new AgentExecutor chain...
Understanding: The user wants to find a place near 방배동 트릿지 where they can eat delicious meat. I need to search for three blogs from Google, summarize each blog, understand all the summaries, get information from Google Place, select the best place from all the information, generate a fluent sentence to suggest the place I found, and translate the sentence to Korean.

Planing: I will start by searching for blogs that recommend good meat restaurants near 방배동 트릿지.

ThisStep: Use the search_url_tool to find blogs that recommend good meat restaurants near 방배동 트릿지.

Action:
```
{
  "action": "search_url_tool",
  "action_input": "방배동 트릿지 근처 고기 맛집 블로그"
}
```

Observation:['TITLE: 방배동 숨은 고기집 맛집 숙성 돼지고기 전문점 원조부안집 방배점 | LINK: https://blog.naver.com/PostView.nhn?blogId=hjcho429&logNo=223208121302&redirect=Dlog&widgetTypeCall=true | DESCRIPTION: 안녕하세요. 블라초블로그에. 와주셔서 감사합니다. \u200b. 주말 저녁 조금 늦은 시간, 전부터. 궁금했던 고기집에 저녁을 먹으러. 방배동에 왔다.', 'TITLE: [돌짱] 내방역 방배동 돼지고기 맛집

/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'blog.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(




Observation:NAVER 블로그 블라초블로그 블로그 검색 이 블로그에서 검색 공감 댓글 3 공유하기 메뉴 바로가기 본문 바로가기 내 블로그 이웃블로그 블로그 홈 로그인 블라초블로그 블로그 메뉴 프롤로그 블로그 지도 서재 태그 안부 활동정보 블로그 글쓰기 가벼운 글쓰기툴 퀵에디터가 오픈했어요! 낙서장 665개의 글 낙서장목록열기 낙서장 방배동 숨은 고기집 맛집 숙성 돼지고기 전문점 원조부안집 방배점 블라초 ・ 2023. 9. 11. 11:24 URL 복사 이웃추가 본문 기타 기능 공유하기 신고하기 안녕하세요. 블라초블로그에와주셔서 감사합니다.​주말 저녁 조금 늦은 시간, 전부터궁금했던 고기집에 저녁을 먹으러방배동에 왔다. 오늘 소개할 곳은돼지고기 전문점 원조부안집이다. 이 집은 내방역과 방배역 사이에 있는 방배서리풀이편한세상 아파트 길 건너좁은 골목 안 깊숙이 자리하고 있다.평소 늘 손님들로 만석인 집이라 일부러식사 시간대를 훌쩍 넘긴 조금 늦은시간에 방문했는데도 매장 안에손님들로 꽉 차 있었음..
The first blog is about a restaurant called 원조부안집, which is a pork specialty restaurant located between 내방역 and 방배역. The blog mentions that the restaurant is always full of customers, indicating that it is popular.

Planing: I will use the request_url_tool to get the contents of the second blog.

ThisStep: Use the request_url_tool to get the contents of the second blog.

Action:
```
{
  "action": "request_url_tool",
  "action_input": "https://blog.naver.com/tenriver/222946509347"


/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'blog.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Observation: NAVER 블로그 민슐랭의 민슐랭 블로그 검색 이 블로그에서 검색 공감 댓글 0 공유하기 메뉴 바로가기 본문 바로가기 내 블로그 이웃블로그 블로그 홈 로그인 민슐랭의 민슐랭 블로그 메뉴 프롤로그 블로그 지도 서재 태그 안부 활동정보 블로그 글쓰기 가벼운 글쓰기툴 퀵에디터가 오픈했어요! 민슐랭의 민슐랭 1,000개의 글 민슐랭의 민슐랭목록열기 민슐랭의 민슐랭 [돌짱] 내방역 방배동 돼지고기 맛집 강력 추천! 민슐랭 ・ 2023. 4. 5. 11:24 URL 복사 이웃추가 본문 기타 기능 공유하기 신고하기 안녕하세요. 피부과전문의 민슐랭입니다.​오랜만에 맛집 탐방을 다녀왔습니다. ㅎㅎ.​근처에 손님이 오시면 자주 가는 소고기집중 하나가. 돌짱이라는 곳인데요.​이곳은 방배동에 위치해 있어서. 저희 진료소에서도 가깝고. 손님들이 오시면 자주 가는 곳이에요.​돌짱은 돼지고기 전문점인데요.​돼지고기를 구워먹는 곳이지만. 소고기도 팔고 있어요.​돼지고기는 물론이고. 소고기도 정말 맛있어요.​돌짱에서는 돼지고기와 소고기를. 함께 구워먹는 것을 추천드려요.​돼지고기는 삼겹살, 목살, 가브리살을 팔고 있어요.​소고기는 등심, 안심, 채끝살을 팔고 있어요.​돌짱에서는 고기를 구워주는 서비스가 있어요.​고기를 구워주는 분이. 정말 친절하게 고기를 구워주셔서. 맛있게 먹을 수 있었어요.​돌짱에서는 고기 외에도. 다양한 반찬을 제공해주는데요.​반찬도 정말 맛있어요.​특히 김치찌개가. 정말 맛있어요.​돌짱에서는 김치찌개를. 고기와 함께 먹는 것을 추천드려요.​돌짱에서는 고기를 구워주는 서비스가 있어요.​고기를 구워주는 분이. 정말 친절하게 고기를 구워주셔서. 맛있게 먹을 수 있었어요.​돌짱에서는 고기 외에도. 다양한 반찬을 제공해주는데요.​반찬도 정말 맛있어요.​특히 김치찌개가. 정말 맛있어요.​돌짱에서는 김치찌개를. 고기와 함께 먹는 것을 추천드려요.​
Understanding: The second blog is about a restaurant called 돌

'육갑식당 서래마을직영점을 한번 가보시는 걸 추천합니다. 방배동 트릿지 근처에 있어요. 평점이 4.3이고 사용자 평가 수가 262개로 높아, 인기가 많고 맛있는 고기를 제공할 가능성이 크다는 것을 나타냅니다.'